# Paired GCN

Training of two Graph convolutional networks with coupled losses in Pytorch

### TODOs:

- currently only works for multiclass classification; maybe extend multilabel with classifier chains or try other tasks

- unsupervised: try various loss functions

- other architectures (deeper GCN, for example)

- replace GCN with GAE?

- try other loss functions

In [13]:
from __future__ import division
from __future__ import print_function

import time
import argparse
from itertools import product

import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from utils import load_multiclass, accuracy
from utils import get_dual
from utils import sparse_mx_to_torch_sparse_tensor
from utils import log_progress

from sklearn.metrics import f1_score, accuracy_score

from models import GCN

import networkx as nx
import scipy.sparse as sp

def f1(output, labels):
    preds = output.max(1)[1].type_as(labels).data
#     print(preds)
#     print(f1_score(labels, preds))
    return f1_score(labels.data, preds, average="macro")

In [2]:
seed = 20

np.random.seed(seed)
torch.manual_seed(seed)

n_hidden = 16
dropout = 0.5
lr = 1e-2
seed = 42
weight_decay = 5e-4
fastmode = True
n_epochs = 300

Loading other data here

In [19]:
# load multiclass data : make blog_catalog classes

In [20]:
!ls ../data/BlogCatalog-dataset/

data       readme.txt


`adj` - матрица смежности, исходного графа в coo формате

`features` -  матрица

`labels`

`idx_train`

`idx_val`

`idx_test`

In [21]:
adj, features, labels, idx_train, idx_val, idx_test = load_multiclass(path='../data/BlogCatalog-dataset/data/',
                                                                      dataset='blog_catalog')


# adj, features, labels, idx_train, idx_val, idx_test = load_multiclass()
features = torch.FloatTensor(np.identity(adj.shape[0]))

Loading blog_catalog dataset...


In [15]:
# All unimported definitions

def make_incidence_matrix(adj, edges, idx_train):
    """
    Make a (V x E) vertex-edge incidence matrix
    """
    row, col = [], []
    for edge_id in edges:
        a, b = edges[edge_id]
        if a not in idx_train or b not in idx_train:
            continue
        row.append(a); col.append(edge_id)
        row.append(b); col.append(edge_id)
    arr = sp.coo_matrix(([1] * len(row), (row, col)), shape=(adj.shape[0], len(edges)))
    return sparse_mx_to_torch_sparse_tensor(arr)

def make_incidence_matrix_EV(adj, edges, idx_train):
    """
    Make a (E x V) vertex-edge incidence matrix
    """
    row, col = [], []
    for edge_id in edges:
        a, b = edges[edge_id]
        if a not in idx_train or b not in idx_train:
            continue
        col.append(a); row.append(edge_id)
        col.append(b); row.append(edge_id)
    arr = sp.coo_matrix(([1] * len(row), (row, col)), shape=(len(edges), adj.shape[0]))
    return sparse_mx_to_torch_sparse_tensor(arr)

class SparseMM2_GCN(torch.autograd.Function):
    """
    Sparse x dense matrix multiplication with autograd support.

    Implementation by Soumith Chintala:
    https://discuss.pytorch.org/t/
    does-pytorch-support-autograd-on-sparse-matrix/6156/7
    """

    def __init__(self, sparse):
        super(SparseMM2_GCN, self).__init__()
        self.sparse = sparse

    def forward(self, dense):
        return torch.mm(self.sparse.data, dense)

    def backward(self, grad_output):
        grad_input = None
        if self.needs_input_grad[0]:
            grad_input = torch.mm(self.sparse.data.t(), grad_output)
        return grad_input

In [23]:
%%time
# in: adj - scipy sparse matrix
# out: 1) dual adjacency matrix
#  and 2) a dict of all edges

# nx graph representation of the original graph
gr = nx.from_scipy_sparse_matrix(adj).to_directed()
edge_to_id = {edge: i for i, edge in enumerate(gr.edges())}
edges = {i: edge for i, edge in enumerate(gr.edges())}

row, col = [], []

nodes = list(gr.nodes())
for v in log_progress(nodes):
    for n1, n2 in product(gr.neighbors(v), gr.neighbors(v)):
        # {v, n1} and {v, n2} are (undirected) edges
        row.append(edge_to_id[(v, n1)])
        col.append(edge_to_id[(v, n2)])
        row.append(edge_to_id[(v, n2)])
        col.append(edge_to_id[(v, n1)])

CPU times: user 6min 46s, sys: 8.52 s, total: 6min 54s
Wall time: 6min 56s


In [24]:
%%time
del gr
adj_d = sp.coo_matrix(([1] * len(row), (row, col)), 
                      shape=(len(edges), len(edges)))
del row, col

CPU times: user 2min 31s, sys: 1min 4s, total: 3min 36s
Wall time: 4min 4s


In [ ]:
%%time
# at this point, adj_d is a scipy.sparse matrix
adj_d = sparse_mx_to_torch_sparse_tensor(adj_d)

In [ ]:
# d times E x E times V = d times V
commat = make_incidence_matrix_EV(adj, edges, idx_train.numpy())

In [ ]:
# features_d = sparse_mx_to_torch_sparse_tensor(sp.eye(adj_d.shape[0]))
features_d = torch.FloatTensor(np.identity(adj_d.shape[0]))

In [ ]:
adj.shape

In [ ]:
len(edges)

Что нам нужно дальше:

adj, features

adj_d, features_d

labels

Наборы индексов для сплита

надо ли приближать на сплите

commat

In [ ]:
# features = sparse_mx_to_torch_sparse_tensor(sp.eye(adj.shape[0]))
# features = Variable(features)

In [ ]:
# conversions
adj = sparse_mx_to_torch_sparse_tensor(adj)
features, labels = Variable(features), Variable(labels)
features_d, commat = Variable(features_d), Variable(commat)

In [ ]:
labels_np = labels.data.numpy()
edge_labels = Variable(torch.LongTensor( np.array([labels_np[a] == labels_np[b] 
                                                  for a, b in edges.values()], dtype='int') ))

In [15]:
sum(edge_labels)

Variable containing:
 11258
[torch.LongTensor of size 1]

In [18]:
edge_labels

Variable containing:
 1
 1
 1
⋮ 
 1
 1
 1
[torch.LongTensor of size 13264]

In [19]:
## do not do this, causes problems with torch.mm
# adj   = Variable(adj)
# adj_d = Variable(adj_d)

In [ ]:
# Model

model = GCN(nfeat=features.shape[1],
            nhid=n_hidden,
            nclass=labels.max().data[0] + 1,
            dropout=dropout)

model_d = GCN(nfeat=features_d.shape[1],
             nhid=n_hidden,
             nclass=2,
             dropout=dropout)

# Optimizers
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

optimizer_d = optim.Adam(model_d.parameters(),
                       lr=lr, weight_decay=weight_decay)




def multiclass_classification_combined_loss(output, labels, output_d, edge_labels, approx_err,
                                            idx_train, alpha, beta):
    """
    Compute negative log-likelihood loss for multiclass task
    """
    orig_loss = F.nll_loss(output[idx_train], labels[idx_train])
    dual_loss = F.nll_loss(output_d, edge_labels)
    
    return orig_loss + alpha * dual_loss + beta * approx_err


        
alpha = 1e-1
beta = 1e-2
alpha = Variable(torch.FloatTensor([alpha]))
beta = Variable(torch.FloatTensor([beta]))


def train(epoch, verbose=True):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    model_d.train()
    optimizer_d.zero_grad()
    output = model(features, adj)
    output_d = model_d(features_d, adj_d)

    approx_err = ((SparseMM2_GCN(commat)(model.gc1.weight) - model_d.gc1.weight) ** 2).sum()

    loss_train = multiclass_classification_combined_loss(output, labels, output_d, edge_labels,
                                                        approx_err, idx_train, alpha, beta)

    
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    optimizer_d.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    if verbose:
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.data[0]),
              'acc_train: {:.4f}'.format(acc_train.data[0]),
              'loss_val: {:.4f}'.format(loss_val.data[0]),
              'acc_val: {:.4f}'.format(acc_val.data[0]),
              'time: {:.4f}s'.format(time.time() - t))

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    f1_test = f1(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data[0]),
          "accuracy= {:.4f}".format(acc_test.data[0]),
          "f1= {:.4f}".format(f1_test))

# Train model
t_total = time.time()

for epoch in range(300):
    train(epoch, verbose=False)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()


## Сравнение с базовой моделью

In [ ]:
# Base model and optimizer

model_base = GCN(nfeat=features.shape[1],
                nhid=n_hidden,
                nclass=labels.max().data[0] + 1,
                dropout=dropout)

optimizer_base = optim.Adam(model_base.parameters(),
                       lr=lr, weight_decay=weight_decay)

def train_base(epoch, verbose=True):
    t = time.time()
    model_base.train()
    optimizer_base.zero_grad()
    output = model_base(features, adj)
    
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])

    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer_base.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model_base.eval()
        output = model_base(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    if verbose:
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.data[0]),
              'acc_train: {:.4f}'.format(acc_train.data[0]),
              'loss_val: {:.4f}'.format(loss_val.data[0]),
              'acc_val: {:.4f}'.format(acc_val.data[0]),
              'time: {:.4f}s'.format(time.time() - t))

def test_base():
    model_base.eval()
    output = model_base(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    f1_test = f1(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data[0]),
          "accuracy= {:.4f}".format(acc_test.data[0]),
          "f1= {:.4f}".format(f1_test))

# Train model
t_total = time.time()

for epoch in range(100):
    train_base(epoch, verbose=False)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test_base()


## Making features (not sparse, use original layers) with diffusion

In [3]:
from utils import load_graph, dotdict, encode_onehot

def get_data_with_diffusion_features(args_p, args_d, subgraph={'num': None, 'seed': 0}, dataset='cora'):    
    path_to_edgelist = args_p.dataset_str

    args_p.input = 'temp'
    args_p.output = 'temp_out'
    args_d.input = 'temp_d'
    args_d.output = 'temp_d_out'

    # set the seed for labels and train-test split
    np.random.seed(subgraph['seed'])
    
    # save edgelists to temp locations
    if dataset=='bc':
        graph = load_graph(path_to_edgelist, subgraph=subgraph['num'], seed=subgraph['seed'])
        bc_classes = '../data/BlogCatalog-dataset/data/blog_catalog.classes'
        # we have this nodelist, so we can select labels for these nodes
        labels = np.array([np.random.choice(
                            np.where(
                                np.array( list(map(int, line.split())) )
                                )[0],
                            size=1)
                            for line in open(bc_classes)],
                            dtype='int')
            
    if dataset=='cora':
        idx_features_labels = np.genfromtxt("../data/cora/cora.content",
                                        dtype=np.dtype(str))
        idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
        idx_map = {j: i for i, j in enumerate(idx)}
        edges_unordered = np.genfromtxt("../data/cora/cora.cites",
                                            dtype=np.int32)
        edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                         dtype=np.int32).reshape(edges_unordered.shape)
        graph = nx.from_edgelist(edges)
        idx_features_labels = np.genfromtxt("../data/cora/cora.content",
                                        dtype=np.dtype(str))
        labels = encode_onehot(idx_features_labels[:, -1])
        labels = np.where(labels)[1]
    
    ## common part
    
    nodelist = sorted(graph.nodes())

    m = {i: n for i, n in enumerate(nodelist)}
    inv_m = {n: i for i, n in enumerate(nodelist)}
    adj_mat = sp.csr_matrix(nx.to_numpy_matrix(graph, nodelist))

    # train-test split
    n = len(nodelist)
    rng = list(range(n))
    np.random.shuffle(rng)
    idx_train = rng[:int(n*0.6)]
    idx_val = rng[int(n*0.6):int(n*0.8)]
    idx_test = rng[int(n*0.8):]

    # save edgelist graph.edges()
    with open(args_p.input, 'w') as f:
         f.write("\n".join(str(inv_m[a]) + "," + str(inv_m[b]) for (a, b) in graph.edges()))
                                        #if inv_m[a] in idx_train and inv_m[b] in idx_train))
    
    dual_graph = nx.line_graph(graph)
    edgelist = sorted(dual_graph.nodes())
    dual_nodes = {e: i for i, e in enumerate(edgelist)}
    # save edgelist dual_graph.edges()
    with open(args_d.input, 'w') as f:
         f.write("\n".join(str(dual_nodes[a]) + "," + str(dual_nodes[b]) for (a, b) in dual_graph.edges()))
                                                                #if a[0] in b and inv_m[a[0]] in idx_train\
                                                                #or a[1] in b and inv_m[a[1]] in idx_train))
    
    adj_dual = sp.csr_matrix(nx.to_numpy_matrix(dual_graph, edgelist))        
    
    ###############################################################################
    
    args_p.dimensions = graph.number_of_nodes()
    args_d.dimensions = dual_graph.number_of_nodes()
    
    # Obtain features
    #----- Primal
    print('\n\tStage 1: training for features')
    emb_ = get_diff2vec_embeddings(args_p,  verbose=0)
    emb = np.zeros(shape=(graph.number_of_nodes(), args_p.dimensions))
    i2w = emb_.index2word
    for w in i2w:
        emb[int(w), :] = emb_[w]


    print('\n\n\n')
    #----- Dual
    print('\n\tStage 2: training on dual')
    emb_dual_ = get_diff2vec_embeddings(args_d, verbose=0)
    emb_dual = np.zeros(shape=(dual_graph.number_of_nodes(), args_d.dimensions))
    i2w = emb_dual_.index2word
    for w in i2w:
        emb_dual[int(w), :] = emb_dual_[w]

    edge_embs = emb_dual
    enum = {node: num for num, node in enumerate(nodelist)}
    num_nodes = adj_mat.shape[0]
    node_features = np.zeros((num_nodes, args_d.dimensions))
    counts = np.ones(num_nodes)
    for i, edge in enumerate(edgelist):
        # средние по эмбеддингам рёбер, связанных с данной вершиной
        u = enum[edge[0]]; v = enum[edge[1]]
        node_features[u, :] += edge_embs[i, :]
        node_features[v, :] -= edge_embs[i, :]
        counts[u] += 1; counts[v] += 1
    node_features /= counts[:, np.newaxis]
    
    #### return two numpy arrays with features
    ## --> maybe also return sp.sparse adjacency matrices
    ## --> and labels for the sampled subgraph
    ## after that, presumably simply do ->Tensors->Variable (except adj) 
    ##     and feed into the model
    return {'data': (adj_mat, labels, idx_train, idx_val, idx_test, adj_dual),
            'features': (emb, node_features, emb_dual)
           }

In [4]:
from diff2vec import get_diff2vec_embeddings
from copy import copy

# path to edgelist
dataset_str = '../data/BlogCatalog-dataset/data/blog_catalog.edgelist'

n_dim = 16
num_epochs = 200
seed = 2

defaults = {'vertex_set_cardinality': 80, 'num_diffusions': 10,
            'workers': 1, 'type': 'eulerian', 'dimensions': n_dim, # set with different values
            'alpha': 0.025, 'window_size': 5, 'iter': 2,
            'dataset_str': dataset_str}

args_p = dotdict(copy(defaults))
args_d = dotdict(copy(defaults))


dict_res = get_data_with_diffusion_features(args_p, args_d, subgraph={'num': 1000, 'seed': 19}, dataset='bc')

The graph has 10312 nodes and 333983 edges
Sampling 1000-node subgraph from original graph

	Stage 1: training for features





	Stage 2: training on dual


In [5]:
# ->tensors->variables
adj, labels, idx_train, idx_val, idx_test, adj_d = dict_res['data']
node_features, node_features_from_dual, features_d = dict_res['features']

In [6]:
def make_incidence_matrix_EV(adj):
    """
    Make a (V x E) vertex-edge incidence matrix
    No masking (multiclass classif)
    """
    edges = nx.from_scipy_sparse_matrix(adj).edges()
    row, col = [], []
    for edge_id, edge in enumerate(edges):
        a, b = edge
        col.append(a); row.append(edge_id)
        col.append(b); row.append(edge_id)
    arr = sp.coo_matrix(([1] * len(row), (row, col)), shape=(len(edges), adj.shape[0]))
    return sparse_mx_to_torch_sparse_tensor(arr)

commat = make_incidence_matrix_EV(adj)

In [7]:
labels.shape

(10312, 1)

In [8]:
adj    = sparse_mx_to_torch_sparse_tensor(adj)
adj_d  = sparse_mx_to_torch_sparse_tensor(adj_d)
labels = torch.LongTensor(labels)
labels = labels.view(labels.size(0))

idx_train = torch.LongTensor(idx_train)
idx_val   = torch.LongTensor(idx_val)
idx_test  = torch.LongTensor(idx_test)

# features   = torch.FloatTensor(np.hstack([node_features, node_features_from_dual]))
features   = torch.FloatTensor(node_features)
features_d = torch.FloatTensor(features_d)

In [9]:
labels = Variable(labels)
features, features_d = Variable(features), Variable(features_d)
commat = Variable(commat)

In [10]:
labels_np = labels.data.numpy()
edge_labels = Variable(torch.LongTensor( np.array([labels_np[a] == labels_np[b] 
                                                  for a, b in nx.from_numpy_array(adj.to_dense().numpy()).edges()], dtype='int') ))

In [11]:
print(adj.shape, adj_d.shape)
print(features.shape)
print(features_d.shape, adj_d.shape)

torch.Size([1000, 1000]) torch.Size([3487, 3487])
torch.Size([1000, 1000])
torch.Size([3487, 3487]) torch.Size([3487, 3487])


In [16]:
# Model

model = GCN(nfeat=features.shape[1],
            nhid=n_hidden,
            nclass=labels.max().data[0] + 1,
            dropout=dropout)

model_d = GCN(nfeat=features_d.shape[1],
             nhid=n_hidden,
             nclass=2,
             dropout=dropout)

# Optimizers
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

optimizer_d = optim.Adam(model_d.parameters(),
                       lr=lr, weight_decay=weight_decay)




def multiclass_classification_combined_loss(output, labels, output_d, edge_labels, approx_err,
                                            idx_train, alpha, beta):
    """
    Compute negative log-likelihood loss for multiclass task
    """
    orig_loss = F.nll_loss(output[idx_train], labels[idx_train])
    dual_loss = F.nll_loss(output_d, edge_labels)
    
    return orig_loss + alpha * dual_loss + beta * approx_err


        
alpha = 1e-1
beta = 1e-2
alpha = Variable(torch.FloatTensor([alpha]))
beta = Variable(torch.FloatTensor([beta]))


def train(epoch, verbose=True):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    model_d.train()
    optimizer_d.zero_grad()
    output = model(features, adj)
    output_d = model_d(features_d, adj_d)

    approx_err = ((SparseMM2_GCN(commat)(model.gc1.weight) - model_d.gc1.weight) ** 2).sum()

    loss_train = multiclass_classification_combined_loss(output, labels, output_d, edge_labels,
                                                        approx_err, idx_train, alpha, beta)

    
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    optimizer_d.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    if verbose:
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.data[0]),
              'acc_train: {:.4f}'.format(acc_train.data[0]),
              'loss_val: {:.4f}'.format(loss_val.data[0]),
              'acc_val: {:.4f}'.format(acc_val.data[0]),
              'time: {:.4f}s'.format(time.time() - t))

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    f1_test = f1(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data[0]),
          "accuracy= {:.4f}".format(acc_test.data[0]),
          "f1= {:.4f}".format(f1_test))


# Train model
t_total = time.time()

for epoch in range(200):
    train(epoch, verbose=False)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Optimization Finished!
Total time elapsed: 69.1271s
Test set results: loss= 3.2592 accuracy= 0.1600 f1= 0.0089


/Users/ksk/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy 0.1650

In [17]:
## uncomment this to replace with dummy features
features = Variable(torch.FloatTensor(np.identity(adj.shape[0])))

In [18]:
# Base model and optimizer

model_base = GCN(nfeat=features.shape[1],
                nhid=n_hidden,
                nclass=labels.max().data[0] + 1,
                dropout=dropout)

optimizer_base = optim.Adam(model_base.parameters(),
                       lr=lr, weight_decay=weight_decay)

def train_base(epoch, verbose=True):
    t = time.time()
    model_base.train()
    optimizer_base.zero_grad()
    output = model_base(features, adj)
    
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])

    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer_base.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model_base.eval()
        output = model_base(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    if verbose:
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.data[0]),
              'acc_train: {:.4f}'.format(acc_train.data[0]),
              'loss_val: {:.4f}'.format(loss_val.data[0]),
              'acc_val: {:.4f}'.format(acc_val.data[0]),
              'time: {:.4f}s'.format(time.time() - t))

def test_base():
    model_base.eval()
    output = model_base(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    f1_test = f1(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data[0]),
          "accuracy= {:.4f}".format(acc_test.data[0]),
          "f1= {:.4f}".format(f1_test))

# Train model
t_total = time.time()

for epoch in range(200):
    train_base(epoch, verbose=False)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test_base()


Optimization Finished!
Total time elapsed: 2.0656s
Test set results: loss= 7.5496 accuracy= 0.1400 f1= 0.0371


/Users/ksk/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


* num = 500, seed = 10:

0.11 base+dummy features, 0.18 diffusion features and dual

Original self-contained piece

In [32]:
# adj, features, labels, idx_train, idx_val, idx_test = load_multiclass(path='../data/BlogCatalog-dataset/data/',
#                                                                       dataset='blog_catalog')


del GCN
from models import GCN

adj, features, labels, idx_train, idx_val, idx_test = load_multiclass()
features = torch.FloatTensor(np.identity(adj.shape[0]))
adj = sparse_mx_to_torch_sparse_tensor(adj)

model = GCN(nfeat=features.shape[1],
            nhid=n_hidden,
            nclass=labels.max() + 1,
            dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

features, labels = Variable(features), Variable(labels)


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data[0]),
          'acc_train: {:.4f}'.format(acc_train.data[0]),
          'loss_val: {:.4f}'.format(loss_val.data[0]),
          'acc_val: {:.4f}'.format(acc_val.data[0]),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data[0]),
          "accuracy= {:.4f}".format(acc_test.data[0]))


# Train model
t_total = time.time()
for epoch in range(500):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Loading cora dataset...


/Users/ksk/Desktop/sem2/graph_embeddings/MAIN/models.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Epoch: 0001 loss_train: 1.9504 acc_train: 0.1310 loss_val: 1.9658 acc_val: 0.1100 time: 0.0278s
Epoch: 0002 loss_train: 1.9393 acc_train: 0.1320 loss_val: 1.9620 acc_val: 0.1150 time: 0.0209s
Epoch: 0003 loss_train: 1.9202 acc_train: 0.2020 loss_val: 1.9424 acc_val: 0.1450 time: 0.0212s
Epoch: 0004 loss_train: 1.9105 acc_train: 0.2740 loss_val: 1.9264 acc_val: 0.2600 time: 0.0204s
Epoch: 0005 loss_train: 1.8961 acc_train: 0.2970 loss_val: 1.9154 acc_val: 0.2900 time: 0.0175s
Epoch: 0006 loss_train: 1.8874 acc_train: 0.3170 loss_val: 1.9025 acc_val: 0.3100 time: 0.0188s
Epoch: 0007 loss_train: 1.8727 acc_train: 0.3150 loss_val: 1.8913 acc_val: 0.3050 time: 0.0186s
Epoch: 0008 loss_train: 1.8587 acc_train: 0.3140 loss_val: 1.8813 acc_val: 0.3100 time: 0.0167s
Epoch: 0009 loss_train: 1.8534 acc_train: 0.3140 loss_val: 1.8714 acc_val: 0.3150 time: 0.0154s
Epoch: 0010 loss_train: 1.8335 acc_train: 0.3150 loss_val: 1.8571 acc_val: 0.3100 time: 0.0178s
Epoch: 0011 loss_train: 1.8193 acc_train

KeyboardInterrupt: 

Without node features:

Accuracy (6-class?) 0.66 this (300 epochs) vs 0.61 baseline (200 epochs)

with node features about the same (weaker data then auxilary information)

# Ссылки


* [PyGCN](https://github.com/tkipf/pygcn/tree/master/pygcn)

* [GAE in Pytorch](https://github.com/vmasrani/gae_in_pytorch/blob/master/)

# Trashbin

In [ ]:
# def make_common_matrix(adj, edges, idx_train):
#     print('Starting')
#     arr = np.zeros(shape=(adj.shape[0], len(edges)))
#     print('Matrix created')
#     for edge_id in edges:
#         a, b = edges[edge_id]
#         if a not in idx_train or b not in idx_train:
#             continue
#         arr[a, edge_id] = 1
#         arr[b, edge_id] = 1
#     print('Returning')
#     return torch.FloatTensor(arr)



## Previous version:
# gr = nx.from_scipy_sparse_matrix(adj)
# sub_nodes = np.random.choice(gr.nodes(), replace=0, size=int(len(gr.nodes()) * 0.4))
# subgraph = gr.subgraph(sub_nodes)
# adj_d, edges = get_dual(subgraph, sparse=False)
# edges = {i: n for i, n in enumerate(edges)}
# adj_d = torch.FloatTensor(adj_d)





# if unsupervised, also use a decoder to compute loss
# decoder = InnerProductDecoder(dropout)
# def unsupervised_combined_loss(output, output_d, approx_err,
#                                alpha, beta):
#     """
#     Compute negative log-likelihood loss for multiclass task
#     """
#     recovered_adj = decoder(output)
#     recovered_adj_d = decoder(output_d)
#     orig_loss = (SparseDiff_GCN(adj)(recovered_adj) ** 2).sum()
#     dual_loss = (SparseDiff_GCN(adj_d)(recovered_adj_d) ** 2).sum()

#     return orig_loss + alpha * dual_loss + beta * approx_err





    #loss_train = F.nll_loss(output[idx_train], labels[idx_train]) + \
    #             alpha * F.nll_loss(output_d, edge_labels) + \
    #             beta * approx_err

In [ ]:
# import math
# import torch
# from torch.nn.parameter import Parameter
# from torch.nn.modules.module import Module
# from torch.autograd import Variable

# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable

# from utils import get_subsampler, make_sparse
# from utils import eval_gae_lp, plot_results
# from preprocessing import preprocess_train_test_split

# from dist import weighted_bernoulli

# from collections import defaultdict


# class SparseMM_GCN(torch.autograd.Function):
#     """
#     Sparse x dense matrix multiplication with autograd support.

#     Implementation by Soumith Chintala:
#     https://discuss.pytorch.org/t/
#     does-pytorch-support-autograd-on-sparse-matrix/6156/7
#     """

#     def __init__(self, sparse):
#         super(SparseMM_GCN, self).__init__()
#         self.sparse = sparse

#     def forward(self, dense):
#         return torch.mm(self.sparse, dense)

#     def backward(self, grad_output):
#         grad_input = None
#         if self.needs_input_grad[0]:
#             grad_input = torch.mm(self.sparse.t(), grad_output)
#         return grad_input


class SparseMM2_GCN(torch.autograd.Function):
    """
    Sparse x dense matrix multiplication with autograd support.

    Implementation by Soumith Chintala:
    https://discuss.pytorch.org/t/
    does-pytorch-support-autograd-on-sparse-matrix/6156/7
    """

    def __init__(self, sparse):
        super(SparseMM2_GCN, self).__init__()
        self.sparse = sparse

    def forward(self, dense):
        return torch.mm(self.sparse.data, dense)

    def backward(self, grad_output):
        grad_input = None
        if self.needs_input_grad[0]:
            grad_input = torch.mm(self.sparse.data.t(), grad_output)
        return grad_input


# class SparseDiff_GCN(torch.autograd.Function):
#     """
#     Sparse x dense matrix substraction with autograd support.
#     """

#     def __init__(self, sparse):
#         super(SparseDiff_GCN, self).__init__()
#         self.sparse = sparse

#     def forward(self, dense):
#         return self.sparse.to_dense() - dense

#     def backward(self, grad_output):
#         grad_input = None
#         if self.needs_input_grad[0]:
#             grad_input = self.sparse.to_dense().t() - grad_output
#         return grad_input


# class GraphConvolution_GCN(Module):
#     """
#     Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
#     """

#     def __init__(self, in_features, out_features, bias=True):
#         super(GraphConvolution_GCN, self).__init__()
#         self.in_features = in_features
#         self.out_features = out_features
#         self.weight = Parameter(torch.Tensor(in_features, out_features))
#         if bias:
#             self.bias = Parameter(torch.Tensor(out_features))
#         else:
#             self.register_parameter('bias', None)
#         self.reset_parameters()

#     def reset_parameters(self):
#         stdv = 1. / math.sqrt(self.weight.size(1))
#         self.weight.data.uniform_(-stdv, stdv)
#         if self.bias is not None:
#             self.bias.data.uniform_(-stdv, stdv)

#     def forward(self, input=None, adj=None, use_input=True):
#         if not use_input:
#             support = self.weight
#         else:
#             support = torch.mm(input, self.weight)
#         output = SparseMM_GCN(adj)(support)
#         if self.bias is not None:
#             return output + self.bias
#         else:
#             return output

#     def __repr__(self):
#         return self.__class__.__name__ + ' (' \
#                + str(self.in_features) + ' -> ' \
#                + str(self.out_features) + ')'


# class InnerProductDecoder(Module):
#     """Decoder for using inner product for prediction."""
#     def __init__(self, dropout):
#         super(InnerProductDecoder, self).__init__()
#         self.dropout = dropout
#         self.sigmoid = nn.Sigmoid()
#         self.fudge = 1e-7

#     def forward(self, z):
#         z = F.dropout(z, self.dropout, training=self.training)
#         adj = (self.sigmoid(torch.mm(z, z.t())) + self.fudge) * (1 - 2 * self.fudge)
#         return adj


# ########################################################################################
# #                                                                                      #
# #######################   GCN - graph convolutional network   ##########################
# #                                                                                      #
# ########################################################################################

# class GCN(Module):
#     def __init__(self, nfeat, nhid, nclass, dropout):
#         super(GCN, self).__init__()

#         self.gc1 = GraphConvolution_GCN(nfeat, nhid)
#         self.gc2 = GraphConvolution_GCN(nhid, nclass)
#         self.dropout = dropout

#     def forward(self, adj):
#         # x - feature matrix?
#         x = F.relu(self.gc1(adj=adj, use_input=False))
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = self.gc2(x, adj)
#         return F.log_softmax(x, dim=1)
